# Historical Bundesliga Match Data Scraping and Integration

Welcome to our Jupyter Notebook focused on scraping historical Bundesliga match data from the past four seasons. In this project, we gather data from https://fbref.com/de/wettbewerbe/20/Bundesliga-Statistiken, which provides comprehensive statistics on Bundesliga matches. Additionally, we incorporate stadium location data from a CSV file obtained from https://github.com/jokecamp/FootballData.git. This extra step is crucial as the original match data lacks specific geographical coordinates, only indicating whether matches were home or away. Our goal is to merge these datasets to enrich our analysis and subsequently store the combined data in a MySQL database for further analysis. This notebook will guide you through the processes of data collection, integration, and storage.

## 1. Setup

In this section of our notebook, we will establish the foundation necessary for our web scraping and data handling activities. Here, we'll import all the libraries essential for fetching and processing web data, handling data structures, and performing data analysis. These libraries include **'requests'** for sending HTTP requests, **'BeautifulSoup'** from **'bs4'** for parsing HTML content, and **'pandas'** along with **'numpy'** for data manipulation and analysis.

In [ ]:
# Import the necessary libraries for web scraping and data manipulation.
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

We will also define the specific URL that contains the historical data for Bundesliga standings. This URL will serve as our primary data source from which we will scrape the necessary information. 

In [ ]:
# Define the URL for the Bundesliga standings page, which contains the data we want to scrape.
bundesliga_standings_url = 'https://fbref.com/en/comps/20/Bundesliga-Stats'

Additionally, we'll execute an initial HTTP GET request to retrieve the HTML content of the page, ensuring we have access to the data needed for our subsequent scraping processes.

In [ ]:
# Send a GET request to the Bundesliga standings URL to fetch the HTML content of the page.
data = requests.get(bundesliga_standings_url)

## 2. Parsing HTML Links with Beautiful Soup

In this chapter, we will delve into the specifics of extracting data from the HTML content we previously fetched. Using Beautiful Soup, a powerful library for parsing HTML and XML documents, we will navigate the structure of the webpage and isolate the data crucial for our project—specifically, the links to team pages. This process involves identifying and parsing HTML tables where team data is listed, extracting hyperlink elements, and formatting these links for further use. The steps outlined here are key for efficiently gathering the detailed team-specific data needed for our analysis.

In [ ]:
# Initialize a BeautifulSoup object with the HTML content to enable parsing.
soup = BeautifulSoup(data.text)

In [ ]:
# Use BeautifulSoup's select() method to find the first occurrence of a table with the class 'stats_table'.
# This table contains the Bundesliga standings information.
standings_table = soup.select('table.stats_table')[0]

In [ ]:
# Extract all anchor tags (<a>) within the standings table to find relevant hyperlinks.
links = standings_table.find_all('a')

In [ ]:
# Extract the 'href' attribute from each anchor tag to gather the individual URLs.
links = [l.get("href") for l in links]

In [ ]:
# Filter out URLs to only include those containing '/squads/', which are links to team pages.
links = [l for l in links if '/squads/' in l]

In [ ]:
# Since the extracted URLs are relative, prepend the base URL 'https://fbref.com' to each to form complete URLs.
team_urls = [f"https://fbref.com{l}" for l in links]

## 3. Retrieve Match Stats for Bayer Leverkusen 

In this chapter, we will focus on extracting detailed match statistics for Bayer Leverkusen, one of the teams in the Bundesliga. By leveraging the URL we obtained in the previous steps, we can access and scrape the specific team page on fbref.com. This process involves sending a HTTP request to the team's page, retrieving the HTML content, and parsing it to extract data about each game they played.

### 3.1 Extract Match Stats using Pandas and Requests

In [ ]:
# Send a GET request to the first URL from our list, which corresponds to Bayer Leverkusen's page.
data = requests.get(team_urls[0])

In [ ]:
# Utilize Pandas to parse the HTML data and extract the table specifically containing "Scores & Fixtures".
# This table includes detailed statistics for each match played by Bayer Leverkusen.
# The read_html function returns a list of DataFrames, where we select the first one that matches our criteria.
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

### 3.2 Get Match Shooting Stats with Request and Pandas

We've secured the primary match stats, yet we require a deeper dive into the gameplay details. This deeper analysis can be found on the shooting page, where stats such as total shots, shots on target, and additional metrics like the number of free kicks and penalties are presented. Our first task is to pinpoint the URL of the shooting page, starting from the scores and fixtures page.

In [ ]:
# We parse the HTML content we retrieved using BeautifulSoup.
soup = BeautifulSoup(data.text)

In [ ]:
# We look for all the 'a' tags to extract hyperlinks from the HTML content.
links = soup.find_all('a')

In [ ]:
# Extracting the 'href' attribute from each link to get the actual URLs.
links = [l.get("href") for l in links]

In [ ]:
# We filter out the list of URLs to keep only the links that contain the reference to 'all_comps/shooting/'.
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
# Making another GET request, this time to the first link of the shooting statistics page.
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
# We use pandas to read the HTML content, specifically looking for the table with 'Shooting' stats.
# The function returns a list, so we access the first table with shooting stats using [0].
shooting = pd.read_html(data.text, match="Shooting")[0]

### 3.3 Cleaning and Merging Scraped Data With Pandas

We observe that the DataFrame displays a multi-level index as we look at the first five rows with the head method. However, in this case, the second level of indices doesn't provide much value. Multi-level indices are often unnecessary for our purposes in pandas. We're going to simplify the DataFrame by removing one level of the index. You can recognize the presence of two index levels by the bolded rows, which suggest multiple header rows. After removing an index level, we'll revisit the shooting stats and confirm the adjustment by checking the head of the DataFrame again.

In [ ]:
# Dropping the top level of the multi-index in the shooting DataFrame's columns for clarity.
shooting.columns = shooting.columns.droplevel()

In [ ]:
# Merging the 'matches' DataFrame with the relevant columns from the 'shooting' DataFrame,
# aligning them by the 'Date' column to combine the match and shooting stats.
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

Following the merge and cleanup process in our notebook, we now have a tidy DataFrame for Bayer Leverkusen's 2023-2024 Bundesliga season. It presents a consolidated view of each match's details along with the shooting statistics, setting us up perfectly for further analysis specific to Bayer Leverkusen's performance.

## 4. Scrapping Data for Multiple Seasons and Teams with a Loop

With our methodology proven successful for Bayer Leverkusen's 2023-2024 season, we're now set to scale up our data scraping process. We'll apply the same procedure to extract and clean data for all teams across the previous four Bundesliga seasons. This comprehensive collection of data will allow us to perform an extensive analysis on the trends and patterns over multiple seasons, enhancing the robustness of our predictive models and insights into the league's dynamics.

In [ ]:
# Create a list of years in reverse order, starting from 2024 to 2021, to iterate through past seasons.
years = list(range(2024, 2020, -1))
years

In [ ]:
# Initialize an empty list to store all the match data we will scrape.
all_matches = []

In [ ]:
standing_url = 'https://fbref.com/en/comps/20/Bundesliga-Stats'

In [ ]:
# Import the time module for pacing our requests to avoid overloading the server.
import time

# Loop through each year to scrape data season by season.
for year in years:
    data = requests.get(standing_url)# Fetch the standings page for the current year in the loop.
    soup = BeautifulSoup(data.text) # Parse the fetched data using BeautifulSoup.
    standings_table = soup.select("table.stats_table")[0] # Select the table that contains the standings.
    
    links = [l.get("href") for l in standings_table.find_all('a')] # Extract all links from the standings table.
    links = [l for l in links if '/squads/' in l] # Filter out links to team pages specifically.
    team_urls = [f"https://fbref.com{l}" for l in links] # Create full URLs from the filtered links.
    
    previous_season = soup.select("a.prev")[0].get("href")
    standing_url = f"https://fbref.com{previous_season}"
    
    # Iterate through each team URL to scrape individual team data.
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") # Extract the team name from the URL.
        
        # Scrape the team's match data.
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        # Parse the team page to extract the shooting data.
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        # Fetch the shooting data page.
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel() # Drop the top level of multi-index from the columns.
        
        # Attempt to merge match stats with shooting stats for each team.
        # If shooting stats are missing and a ValueError occurs, skip this team and move to the next.
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
            
        # ilter matches for Bundesliga and include Season and Team columns
        team_data = team_data[team_data["Comp"] == "Bundesliga"]
        team_data["Season"] = year
        team_data["Team"] = team_name

        # Append the matches for the team to the list
        all_matches.append(team_data)
        
        time.sleep(3)

The list all_matches is a collection of dataframes. Each team_data dataframe is added to that list. Now we need to combine all of the individual dataframes into a single dataframe. We can use the concat function in pandas to do this.

In [ ]:
# Merging the data
match_df = pd.concat(all_matches)

In [ ]:
# Convert column header to lower case
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
# Drop unnecessary column
match_df = match_df.drop('match report', axis=1)

In [ ]:
match_df.head()

In [ ]:
match_df.info()

## 5. Save Match Data in MySQL

After successfully retrieving and merging the match and shooting data for each team, the next critical step is to store this enriched dataset in a MySQL database. Storing the data in MySQL not only secures it for long-term access but also facilitates efficient retrieval for future analysis and reporting. This process involves setting up the database schema, ensuring data integrity, and executing the database commands to load the data.

In [ ]:
# Import required libraries
import mysql.connector
from mysql.connector import Error

In [ ]:
# SQL query to create a 'matches' table in the database if it doesn't already exist.
# Each column is defined with an appropriate data type to ensure data integrity.
table_creation_query = """
         CREATE TABLE IF NOT EXISTS matches (
             id INT AUTO_INCREMENT PRIMARY KEY,
             date VARCHAR(255),
             time VARCHAR(255),
             comp VARCHAR(255),
             round VARCHAR(255),
             day VARCHAR(255),
             venue VARCHAR(255),
             result VARCHAR(255),
             gf FLOAT,
             ga FLOAT,
             opponent VARCHAR(255),
             xg FLOAT,
             xga FLOAT,
             poss FLOAT,
             attendance FLOAT,
             captain VARCHAR(255),
             formation VARCHAR(255),
             referee VARCHAR(255),
             notes VARCHAR(255),
             sh FLOAT,
             sot FLOAT,
             dist FLOAT,
             fk INT,
             pk INT,
             pkatt INT,
             season INT,
             team VARCHAR(255)
         );
        """

In [ ]:
# SQL query to insert data into each column of the 'matches' table
insert_query = """
INSERT INTO matches (
    date, time, comp, round, day, venue, result, gf, ga, opponent,
    xg, xga, poss, attendance, captain, formation, referee, notes,
    sh, sot, dist, fk, pk, pkatt, season, team
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Prepare the data for insertion by converting the DataFrame row to a list
values = list(row[[
    'date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga', 'opponent',
    'xg', 'xga', 'poss', 'attendance', 'captain', 'formation', 'referee', 'notes',
    'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'season', 'team'
]])

In [ ]:
# Try to establish a connection with the MySQL database using the provided credentials.
try:
    connection = mysql.connector.connect(
        user='root',
        password='abcabc123',
        host='localhost',
        database='adsfootball'
    )

    # Check if the connection was successful    
    if connection.is_connected():
        
        # Create a cursor object to interact with the database
        cursor = connection.cursor()

        # Clear any existing table to avoid duplication of structure
        cursor.execute("DROP TABLE IF EXISTS matches")

        # Execute the SQL query to create a new 'matches' table as defined earlier
        cursor.execute(table_creation_query)
        
        # Convert any NaN values in the DataFrame to None for SQL compatibility
        match_df = match_df.fillna(np.nan).replace([np.nan], [None])
        
        # Insert DataFrame records into the SQL table one row at a time
        for i, row in match_df.iterrows():
            # Attempt to execute the insert query with the prepared values
            try:
                cursor.execute(insert_query, values)
            except Error as e:
                # Log any errors that occur during the insert operation
                print(f"Error inserting data: {e}")
                print(f"Row with error: {values}")
            
        # Commit all changes to the database to ensure data is saved
        connection.commit()
        print('Data successfully commited.')

# Catch and print any errors encountered during database connection or query execution
except Error as e:
    print(f"Error while connecting to MySQL: {e}")
    
# Ensure the database connection is closed properly
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

## 6. Get Stadium Data


In our football matches dataset, the specific locations where the games are held are not included. To remedy this, we will utilize a GitHub repository that hosts a CSV file with detailed stadium data, available at: https://github.com/jokecamp/FootballData.git. Our objective is to retrieve this stadium data and integrate it into our MySQL database to enrich our dataset with accurate location information for each match. This step will enhance our analysis capabilities by allowing us to consider the geographical context of the games.

In [ ]:
# Load the CSV file containing stadium data with GPS coordinates into a Pandas DataFrame.
stadium_data_df = pd.read_csv("stadiums-with-GPS-coordinates.csv")

In [ ]:
# Filter the DataFrame to include only stadiums located in Germany.
# This refinement is necessary as our analysis focuses on Bundesliga matches, which are played in Germany.
stadium_data_df = stadium_data_df[stadium_data_df["Country"]=="Germany"]
stadium_data_df

In [ ]:
stadium_data_df.info()

## 7. Save Stadium Data in MySQL

In [ ]:
# Define an SQL query to create a new table named 'stadiums' in the database if it does not already exist.
table_creation_query_stadium = """
CREATE TABLE IF NOT EXISTS stadiums (
    id INT AUTO_INCREMENT PRIMARY KEY,
    team VARCHAR(255),
    fdcouk VARCHAR(255),
    city VARCHAR(255),
    stadium VARCHAR(255),
    capacity INT,
    latitude FLOAT,
    longitude FLOAT,
    country VARCHAR(255)
);
"""

In [ ]:
# Define an SQL insert statement for adding new records into the 'stadiums' table.
insert_statement_stadium = """
INSERT INTO stadiums (team, fdcouk, city, stadium, capacity, latitude, longitude, country)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Attempt to establish a connection to the MySQL database using specified credentials and database information.
try:
    connection = mysql.connector.connect(
        host='localhost',      
        database='adsfootball',  
        user='root',           
        password='abcabc123'   
    )

    # Check if the database connection was successful
    if connection.is_connected():
        # Create a cursor object using the connection
        cursor = connection.cursor()

         # Drop any existing table named 'stadiums' to prevent duplicate data issues
        table_drop_query = "DROP TABLE IF EXISTS stadiums;"
        cursor.execute(table_drop_query)
        
        # Create a new 'stadiums' table based on predefined SQL schema
        cursor.execute(table_creation_query_stadium)

        # Insert data into the 'stadiums' table from the DataFrame row by row
        for index, row in stadium_data_df.iterrows():
            # Prepare data tuple for insertion
            data_tuple = (row['Team'], row['FDCOUK'], row['City'], row['Stadium'], 
                          row['Capacity'], row['Latitude'], row['Longitude'], row['Country'])
            cursor.execute(insert_statement_stadium, data_tuple)
        
        # Commit all changes to the database to ensure data is saved
        connection.commit()
        print('Data successfully commited.')
        
        # Close the cursor to release database resources
        cursor.close()
        
except Error as e:
    print(f"Error: {e}")

finally:
    # Ensure the database connection is closed properly
    if connection and connection.is_connected():
        connection.close()
        print("MySQL connection is closed")